# UFC Data Exploration

Este notebook contiene la exploración inicial de los datos scrapeados de UFC, incluyendo:
- Carga y visualización inicial de los datasets
- Análisis de la estructura de los datos
- Estadísticas descriptivas básicas
- Visualizaciones exploratorias

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Carga de Datos

In [ ]:
# Cargar los datasets
fights_df = pd.read_csv('data/raw/raw_fights.csv')
events_df = pd.read_csv('data/raw/raw_events.csv')
fighters_df = pd.read_csv('data/raw/raw_fighters.csv')


print("Datasets cargados exitosamente!")
print(f"Fights: {fights_df.shape[0]} filas, {fights_df.shape[1]} columnas")
print(f"Events: {events_df.shape[0]} filas, {events_df.shape[1]} columnas")
print(f"Fighters: {fighters_df.shape[0]} filas, {fighters_df.shape[1]} columnas")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/raw_fights.csv'

## 2. Estructura de los Datos

In [ ]:
# Información general de cada dataset
print("=== FIGHTS DATASET ===")
print(fights_df.info())
print("\nPrimeras 3 filas:")
display(fights_df.head(3))

In [ ]:
print("=== EVENTS DATASET ===")
print(events_df.info())
print("\nPrimeras 3 filas:")
display(events_df.head(3))

In [ ]:
print("=== FIGHTERS DATASET ===")
print(fighters_df.info())
print("\nPrimeras 3 filas:")
display(fighters_df.head(3))

## 3. Análisis de Valores Únicos y Faltantes

In [ ]:
# Función para analizar valores faltantes y únicos
def analyze_missing_and_unique(df, name):
    print(f"\n=== {name.upper()} ANALYSIS ===")
    
    missing_data = pd.DataFrame({
        'Column': df.columns,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
        'Unique_Values': [df[col].nunique() for col in df.columns],
        'Data_Type': df.dtypes
    })
    
    # Ordenar por porcentaje de valores faltantes
    missing_data = missing_data.sort_values('Missing_Percentage', ascending=False)
    
    display(missing_data)
    
    return missing_data

# Analizar cada dataset
fights_analysis = analyze_missing_and_unique(fights_df, 'Fights')
events_analysis = analyze_missing_and_unique(events_df, 'Events')
fighters_analysis = analyze_missing_and_unique(fighters_df, 'Fighters')

## 4. Estadísticas Descriptivas

In [ ]:
# Estadísticas descriptivas para columnas numéricas en fights
print("=== FIGHTS - ESTADÍSTICAS NUMÉRICAS ===")
numeric_cols_fights = fights_df.select_dtypes(include=[np.number]).columns
if len(numeric_cols_fights) > 0:
    display(fights_df[numeric_cols_fights].describe())
else:
    print("No hay columnas numéricas detectadas automáticamente")
    # Intentar convertir algunas columnas que deberían ser numéricas
    numeric_like_cols = ['round', 'kd1', 'kd2']
    for col in numeric_like_cols:
        if col in fights_df.columns:
            try:
                fights_df[col] = pd.to_numeric(fights_df[col], errors='coerce')
            except:
                pass
    
    numeric_cols_fights = fights_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols_fights) > 0:
        display(fights_df[numeric_cols_fights].describe())

In [ ]:
# Estadísticas descriptivas para fighters
print("=== FIGHTERS - ESTADÍSTICAS NUMÉRICAS ===")
numeric_cols_fighters = fighters_df.select_dtypes(include=[np.number]).columns
if len(numeric_cols_fighters) > 0:
    display(fighters_df[numeric_cols_fighters].describe())
else:
    print("Convirtiendo columnas numéricas...")
    # Columnas que deberían ser numéricas
    numeric_like_cols = ['wins', 'losses', 'defeats', 'slpm', 'str_acc', 'sapm', 'str_def', 'td_avg', 'td_acc', 'td_def', 'sub_avg']
    for col in numeric_like_cols:
        if col in fighters_df.columns:
            # Limpiar porcentajes y convertir
            fighters_df[col] = fighters_df[col].astype(str).str.replace('%', '').replace('', np.nan)
            fighters_df[col] = pd.to_numeric(fighters_df[col], errors='coerce')
    
    numeric_cols_fighters = fighters_df.select_dtypes(include=[np.number]).columns
    display(fighters_df[numeric_cols_fighters].describe())

## 5. Análisis de Categorías

In [ ]:
# Análisis de weight classes
if 'weight_class' in fights_df.columns:
    print("=== DISTRIBUCIÓN DE WEIGHT CLASSES ===")
    weight_class_counts = fights_df['weight_class'].value_counts()
    print(weight_class_counts)
    
    # Gráfico de barras
    plt.figure(figsize=(12, 6))
    weight_class_counts.plot(kind='bar', color='skyblue', edgecolor='navy')
    plt.title('Distribución de Peleas por Categoría de Peso', fontsize=14, fontweight='bold')
    plt.xlabel('Categoría de Peso')
    plt.ylabel('Número de Peleas')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Análisis de métodos de finalización
if 'method' in fights_df.columns:
    print("\n=== MÉTODOS DE FINALIZACIÓN ===")
    method_counts = fights_df['method'].value_counts()
    print(method_counts)
    
    # Gráfico de pie
    plt.figure(figsize=(10, 8))
    plt.pie(method_counts.values, labels=method_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribución de Métodos de Finalización', fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.show()

In [ ]:
# Análisis de stance (postura) de fighters
if 'stance' in fighters_df.columns:
    print("\n=== DISTRIBUCIÓN DE STANCE ===")
    stance_counts = fighters_df['stance'].value_counts()
    print(stance_counts)
    
    # Gráfico de barras
    plt.figure(figsize=(8, 6))
    stance_counts.plot(kind='bar', color='lightgreen', edgecolor='darkgreen')
    plt.title('Distribución de Stance de los Peleadores', fontsize=14, fontweight='bold')
    plt.xlabel('Stance')
    plt.ylabel('Número de Peleadores')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

## 6. Análisis de Relaciones entre Datasets

In [ ]:
# Verificar relaciones entre datasets
print("=== ANÁLISIS DE RELACIONES ===")

# Events únicos en fights vs events dataset
if 'event_id' in fights_df.columns and 'event_id' in events_df.columns:
    events_in_fights = set(fights_df['event_id'].unique())
    events_in_events = set(events_df['event_id'].unique())
    
    print(f"Eventos únicos en fights: {len(events_in_fights)}")
    print(f"Eventos únicos en events: {len(events_in_events)}")
    print(f"Eventos en common: {len(events_in_fights.intersection(events_in_events))}")
    
    if events_in_fights - events_in_events:
        print(f"Eventos en fights pero no en events: {events_in_fights - events_in_events}")
    if events_in_events - events_in_fights:
        print(f"Eventos en events pero no en fights: {events_in_events - events_in_fights}")

print()

# Fighters únicos en fights vs fighters dataset
if all(col in fights_df.columns for col in ['fighter1_id', 'fighter2_id']) and 'fighter_id' in fighters_df.columns:
    fighters_in_fights = set()
    fighters_in_fights.update(fights_df['fighter1_id'].dropna().unique())
    fighters_in_fights.update(fights_df['fighter2_id'].dropna().unique())
    
    fighters_in_fighters = set(fighters_df['fighter_id'].unique())
    
    print(f"Peleadores únicos en fights: {len(fighters_in_fights)}")
    print(f"Peleadores únicos en fighters: {len(fighters_in_fighters)}")
    print(f"Peleadores en común: {len(fighters_in_fights.intersection(fighters_in_fighters))}")
    
    missing_fighters = fighters_in_fights - fighters_in_fighters
    if missing_fighters:
        print(f"Peleadores en fights pero no en fighters dataset: {len(missing_fighters)}")
        if len(missing_fighters) <= 10:
            print(f"IDs faltantes: {missing_fighters}")

## 7. Análisis Temporal (si aplica)

In [ ]:
# Análisis temporal de eventos
if 'date' in events_df.columns:
    print("=== ANÁLISIS TEMPORAL DE EVENTOS ===")
    
    # Intentar parsear fechas
    events_df['date_parsed'] = pd.to_datetime(events_df['date'], errors='coerce')
    
    if events_df['date_parsed'].notna().sum() > 0:
        # Estadísticas de fechas
        print(f"Rango de fechas: {events_df['date_parsed'].min()} a {events_df['date_parsed'].max()}")
        
        # Distribución por año
        events_df['year'] = events_df['date_parsed'].dt.year
        year_counts = events_df['year'].value_counts().sort_index()
        
        plt.figure(figsize=(12, 6))
        year_counts.plot(kind='line', marker='o', linewidth=2, markersize=8)
        plt.title('Número de Eventos por Año', fontsize=14, fontweight='bold')
        plt.xlabel('Año')
        plt.ylabel('Número de Eventos')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print("No se pudieron parsear las fechas correctamente")
        print("Formatos de fecha encontrados:")
        print(events_df['date'].value_counts().head())

## 8. Resumen de la Exploración

In [ ]:
print("=== RESUMEN DE LA EXPLORACIÓN DE DATOS ===")
print()
print("DATASETS ANALIZADOS:")
print(f"1. Fights: {fights_df.shape[0]} peleas con {fights_df.shape[1]} atributos")
print(f"2. Events: {events_df.shape[0]} eventos con {events_df.shape[1]} atributos")
print(f"3. Fighters: {fighters_df.shape[0]} peleadores con {fighters_df.shape[1]} atributos")
print()

print("PRINCIPALES HALLAZGOS:")
print("- Estructura de datos compleja con múltiples métricas por pelea")
print("- Datos de estadísticas detalladas (striking, grappling, etc.)")
print("- Información de peleadores con métricas de rendimiento")
print("- Datos de eventos con fechas y ubicaciones")
print()

print("PRÓXIMOS PASOS:")
print("1. Data Quality Analysis - Identificar problemas de calidad específicos")
print("2. Data Cleaning - Limpiar y estandarizar los datos")
print("3. Feature Engineering - Crear variables derivadas útiles")
print("4. Análisis más profundo de patrones y correlaciones")

In [ ]:
# Guardar un resumen de los hallazgos
summary_data = {
    'dataset': ['fights', 'events', 'fighters'],
    'rows': [fights_df.shape[0], events_df.shape[0], fighters_df.shape[0]],
    'columns': [fights_df.shape[1], events_df.shape[1], fighters_df.shape[1]],
    'missing_columns': [
        fights_analysis[fights_analysis['Missing_Count'] > 0].shape[0],
        events_analysis[events_analysis['Missing_Count'] > 0].shape[0],
        fighters_analysis[fighters_analysis['Missing_Count'] > 0].shape[0]
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('data_exploration_summary.csv', index=False)
print("Resumen guardado en 'data_exploration_summary.csv'")